# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [14]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np
import bokeh.io
import bokeh.plotting
from bokeh.io import output_notebook
output_notebook()
import bokeh
import hvplot as hv
hv.extension('bokeh')

# Import API key
from api_keys import geoapify_key

Loading BokehJS ...

In [2]:
import cartopy.crs as ccrs
import geoviews as gv
import pyproj

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hermanus,-34.4187,19.2345,13.79,53,29,9.09,ZA,1698532297
1,1,ushuaia,-54.8000,-68.3000,5.81,65,40,6.69,AR,1698532292
2,2,lorengau,-2.0226,147.2712,28.01,82,97,4.60,PG,1698532298
3,3,mandalgovi,45.7625,106.2708,-0.79,62,82,6.24,MN,1698532298
4,4,leninskoye,41.7589,69.3864,17.02,45,75,2.47,KZ,1698532299


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
# %%capture --no-display

# Configure the map plot

map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

map_plot


# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0),:
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna() 

# Display sample data
narrowed_city_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,21,ocean city,39.2776,-74.5746,24.14,70,0,3.20,US,1698532303
70,70,kill devil hills,36.0307,-75.6760,23.32,71,0,1.54,US,1698532316
92,92,bethel,41.3712,-73.4140,24.20,57,0,2.06,US,1698532321
247,247,megarine,33.1920,6.0870,23.08,27,0,2.57,DZ,1698532370
301,301,freeport,40.6576,-73.5832,26.10,68,0,4.12,US,1698532361
309,309,murzuq,25.9155,13.9184,24.19,23,0,3.60,LY,1698532387
311,311,leones,-32.6617,-62.2968,24.47,49,0,4.14,AR,1698532387
314,314,bardai,11.6500,7.2333,23.50,26,0,2.25,NG,1698532388
324,324,chiredzi,-21.0500,31.6667,24.77,54,0,3.38,ZW,1698532392
358,358,moultrie,31.1799,-83.7891,26.87,57,0,2.06,US,1698532406


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]]
hotel_df
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = np.nan
hotel_df


# Display sample data
hotel_df

/var/folders/5c/zj027psx22l2qj3zj7dd6b9c0000gn/T/ipykernel_17252/835596292.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = np.nan


,City,Country,Lat,Lng,Humidity,Hotel Name
21,ocean city,US,39.2776,-74.5746,70,NaN
70,kill devil hills,US,36.0307,-75.6760,71,NaN
92,bethel,US,41.3712,-73.4140,57,NaN
247,megarine,DZ,33.1920,6.0870,27,NaN
301,freeport,US,40.6576,-73.5832,68,NaN
309,murzuq,LY,25.9155,13.9184,23,NaN
311,leones,AR,-32.6617,-62.2968,49,NaN
314,bardai,NG,11.6500,7.2333,26,NaN
324,chiredzi,ZW,-21.0500,31.6667,54,NaN
358,moultrie,US,31.1799,-83.7891,57,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius =  10000
params = {}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{row['Lng']},{row['Lat']},{radius}"
    params["bias"] = f"proximity:{row['Lng']},{row['Lat']}"
    params["limit"] = 2
    params["apiKey"] = geoapify_key
    params["categories"] = "accommodation.hotel"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    with requests.get(url=base_url, params=params) as response:
      # Make and API request using the params dictionaty

      name_address = response.json()
    
    # Convert the API response to JSON format
    # name_address = 
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ocean city - nearest hotel: Watson's Regency Suites
kill devil hills - nearest hotel: Mariner Days Inn & Suites
bethel - nearest hotel: Hampton Inn Danbury
megarine - nearest hotel: No hotel found
freeport - nearest hotel: The Freeport Inn and Marina
murzuq - nearest hotel: فندق باريس - مرزق
leones - nearest hotel: Hotel Parque
bardai - nearest hotel: No hotel found
chiredzi - nearest hotel: Nesbitt Arms Hotel
moultrie - nearest hotel: Hampton
tankara - nearest hotel: No hotel found
sinjar - nearest hotel: No hotel found
yarmouth - nearest hotel: The Inn At Cape Cod
tsiombe - nearest hotel: No hotel found
bang rakam - nearest hotel: No hotel found
bangaon - nearest hotel: Maa Santoshi Lodge
karachi - nearest hotel: Quetta Agha Hotel
puerto penasco - nearest hotel: Hotel Paraíso Del Desierto
saint-pierre - nearest hotel: Tropic Hotel
idri - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
21,ocean city,US,39.2776,-74.5746,70,Watson's Regency Suites
70,kill devil hills,US,36.0307,-75.6760,71,Mariner Days Inn & Suites
92,bethel,US,41.3712,-73.4140,57,Hampton Inn Danbury
247,megarine,DZ,33.1920,6.0870,27,No hotel found
301,freeport,US,40.6576,-73.5832,68,The Freeport Inn and Marina
309,murzuq,LY,25.9155,13.9184,23,فندق باريس - مرزق
311,leones,AR,-32.6617,-62.2968,49,Hotel Parque
314,bardai,NG,11.6500,7.2333,26,No hotel found
324,chiredzi,ZW,-21.0500,31.6667,54,Nesbitt Arms Hotel
358,moultrie,US,31.1799,-83.7891,57,Hampton


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)